In [308]:
import numpy as np
import torch
import pandas as pd
from torch import nn
from torch.utils.data import Dataset, DataLoader

In [456]:
class CustomDataset(Dataset):
    def __init__(self, analyze_ticks, result_ticks, start_ticks, end_ticks, viborka):
        self.data_path = "testingData.csv"
        self.data = []
        df = pd.read_csv(self.data_path, encoding='utf-16', delimiter=";", header=None)
        ticksArray = np.array(df.values)
        if end_ticks>ticksArray.shape[0]:
            end_ticks=ticksArray.shape[0]
        if start_ticks>ticksArray.shape[0]:
            start_ticks=0
        # max_sum = 0;
        
        for idx in np.arange(analyze_ticks-1+start_ticks, end_ticks-result_ticks-1):
            
            analyze_set = np.array([])
            for i in np.arange(analyze_ticks-1, 0, -1):
                analyze_set = np.append(analyze_set, ticksArray[idx-i], 0)
            result_sum = 0
            for i in np.arange(result_ticks):
                result_sum+=ticksArray[idx+i+1][1]
            # if abs(result_sum)>max_sum:
            #     max_sum=abs(result_sum)
            
            if(abs(result_sum)>20):
                self.data.append([analyze_set, result_sum])
            elif np.random.rand()<viborka:
                self.data.append([analyze_set, result_sum])
            # self.data.append([analyze_set, result_sum])
            
        # print(max_sum)
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        analyze_set, result_sum = self.data[idx]
        analyze_tensor = torch.from_numpy(analyze_set).to(torch.float32)
        result_tensor = torch.tensor([result_sum]).to(torch.float32)
        return analyze_tensor, result_tensor

In [486]:
dataset_train = CustomDataset(51,20,0,50000, 0.1)
dataset_test = CustomDataset(51,20,50000, 60000, 0.1)

batch_size0 = 100

train_dataloader = DataLoader(dataset_train, batch_size=batch_size0, shuffle=True)
test_dataloader = DataLoader(dataset_test, batch_size=batch_size0, shuffle=True)


In [479]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(100, 30),
            nn.ReLU(),
            nn.Linear(30, 15),
            nn.ReLU(),
            nn.Linear(15, 10),
            nn.ReLU(),
            nn.Linear(10, 1),
        )
        # print(self.linear_relu_stack[0].weight.dtype)

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [480]:
model = NeuralNetwork()
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=100, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=15, bias=True)
    (3): ReLU()
    (4): Linear(in_features=15, out_features=10, bias=True)
    (5): ReLU()
    (6): Linear(in_features=10, out_features=1, bias=True)
  )
)


In [481]:
learning_rate = 1e-6


In [482]:
def train_loop(dataloader, model, loss_fn, optimizer, log):
    size = len(dataloader.dataset)
    one_time = True
    for batch, (X, y) in enumerate(dataloader):
        
        # Compute prediction and loss
        pred = model(X)
        if one_time:
            one_time= False
            # print(pred)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 30 == 0 and log:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn, min_step, accuracy, log):
    # size = len(dataloader.dataset)
    size = 0
    size1 = 0
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            
            for idx in range(pred.shape[0]):
                if abs(y[idx][0])>min_step:
                    size1 +=1
                if abs(pred[idx][0])>min_step:
                    size +=1
                    if(pred[idx][0]+accuracy>=y[idx][0] and pred[idx][0]-accuracy<=y[idx][0]):
                        correct += 1
    if size!=0 and num_batches!=0:
        test_loss /= num_batches
        correct /= size
    if log:
        print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n Size: {(size):>0.1f}, Size1: {(size1):>0.1f} \n")

In [483]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [487]:


epochs = 10000
for t in range(epochs):
    log = False
    if (t+1)%500==0:
        log = True
        print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer, log)
    test_loop(test_dataloader, model, loss_fn, 20, 7, log)
print("Done!")

Epoch 500
-------------------------------
loss: 269.044678  [  100/ 5455]
loss: 178.163101  [ 3100/ 5455]
Test Error: 
 Accuracy: 0.0%, Avg loss: 252.408975 
 Size: 20.0, Size1: 247.0 

Epoch 1000
-------------------------------
loss: 162.266617  [  100/ 5455]
loss: 182.180740  [ 3100/ 5455]
Test Error: 
 Accuracy: 0.0%, Avg loss: 222.417175 
 Size: 12.0, Size1: 247.0 

Epoch 1500
-------------------------------
loss: 144.290222  [  100/ 5455]
loss: 116.188225  [ 3100/ 5455]
Test Error: 
 Accuracy: 0.0%, Avg loss: 219.295610 
 Size: 4.0, Size1: 247.0 

Epoch 2000
-------------------------------
loss: 141.252975  [  100/ 5455]
loss: 79.427910  [ 3100/ 5455]
Test Error: 
 Accuracy: 0.0%, Avg loss: 219.733035 
 Size: 4.0, Size1: 247.0 

Epoch 2500
-------------------------------
loss: 84.482910  [  100/ 5455]
loss: 74.571632  [ 3100/ 5455]
Test Error: 
 Accuracy: 0.0%, Avg loss: 216.844651 
 Size: 2.0, Size1: 247.0 



KeyboardInterrupt: 

In [454]:
test_loop(test_dataloader, model, loss_fn, 6, log)

Test Error: 
 Accuracy: 100.0%, Avg loss: 190.807506 
 Size: 1.0 



In [353]:
torch.save(model, "model 51 20 v1 perc50.pth")

In [ ]:
model = torch.load("model 51 20 v3 perc6.pth")

In [14]:
epochs=10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    test_loop(test_dataloader, model, loss_fn, 3, True)

Epoch 1
-------------------------------
Test Error: 
 Accuracy: 2.0%, Avg loss: 2344.025424 

Epoch 2
-------------------------------
Test Error: 
 Accuracy: 2.0%, Avg loss: 2341.977581 

Epoch 3
-------------------------------
Test Error: 
 Accuracy: 2.0%, Avg loss: 2342.015881 

Epoch 4
-------------------------------
Test Error: 
 Accuracy: 2.0%, Avg loss: 2341.980488 

Epoch 5
-------------------------------
Test Error: 
 Accuracy: 2.0%, Avg loss: 2342.068672 

Epoch 6
-------------------------------
Test Error: 
 Accuracy: 2.0%, Avg loss: 2341.973897 

Epoch 7
-------------------------------
Test Error: 
 Accuracy: 2.0%, Avg loss: 2341.983285 

Epoch 8
-------------------------------
Test Error: 
 Accuracy: 2.0%, Avg loss: 2342.107387 

Epoch 9
-------------------------------
Test Error: 
 Accuracy: 2.0%, Avg loss: 2341.960891 

Epoch 10
-------------------------------
Test Error: 
 Accuracy: 2.0%, Avg loss: 2341.968734 

